# Testing socket connection between Python code (client) and Warteschlangensimulator (server)

How to test:

* Run this notebook.
* If Java or Warteschlangensimulator is not found or if you want to start Warteschlangensimulator outside this notebook and just want to connect to a running instance, change the settings in "Configuration".

## Import Python modules

In [1]:
from socket_connect import QS_full_connect as QS, QS_socket_connect as QS_socket_only
from model_processor import Model, Statistics, get_example_model

## Configuration

In [2]:
# If Java auto dection is not working, specify the path to the Java runtime environment or development kit here
java_path = None

# If Warteschlangensimulator auto detection is not working, specify the path to Warteschlangensimulator here
simulator_path = None

# If the simulator is already running and is not to be started and stopped by this script,
# enter the port number here. Otherwide let this field at -1.
simulator_already_running_on_port = -1

## Perform connection test

In [3]:
# Load model
xml_model = get_example_model()
# Load custom model: xml_model = Model("filename.xml").get()

# Perform test
if simulator_already_running_on_port > 0:
    print("Connect to already running socket server.")
    qs = QS_socket_only("localhost", port=simulator_already_running_on_port)
    xml_result = qs.run_task(xml_model)
else:
    print("Starting simulator in socket server mode, connect to simulator.")
    with QS(java_path, simulator_path) as qs:
        xml_result = qs.run_task(xml_model)

Starting simulator in socket server mode, connect to simulator.


## Display test result

In [4]:
if type(xml_result) == bytes:
    print("Connection is working.")
    print("Received", len(xml_result), "bytes of statistic data.")
    # Save results: Statistics(xml_result).save("filename.xml")
else:
    print("Sending model or receiving statistics failed.")
    print("Result data type:", type(xml_result))

Connection is working.
Received 289333 bytes of statistic data.


## Example code for running an optimization
(We assume we use "Starting simulator in socket server mode")

In [5]:
# # Load initial model
# model = Model("filename.xml")
# 
# def optimization_step(x, simulator, model):
#     model.set_attr("station", "tag", "attribute", x)  # Define what to change
#     xml_model = model.get()
#     xml_result = simulator.run_task(xml_model)
#     statistics = Statistics(xml_result)
#     y = statistics.get_attr("xmlpath", "attribute")  # Define objective value
#     return y
# 
# with QS(java_path, simulator_path) as qs:
#     # Add optimization loop here
#     x = 123
#     y = optimization_step(x, qs, model)

If you want to use [PyBOBYQA](https://numericalalgorithmsgroup.github.io/pybobyqa/build/html/index.html), you can run the optimization by:

    with QS(java_path, simulator_path) as qs:
        soln = pybobyqa.solve(optimization_step, x0, bounds=(a, b), args=(qs, model))